In [145]:
import pandas as pd
from random import seed, randrange
from csv import reader
from math import sqrt

In [146]:
sonar = pd.read_csv('sonar.csv', header = None)
sonar.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


#### Calculating Splits

In a decision tree, split points are chosen by finding the attribute and the value of that attribute that results in the lowest cost.

Gini is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. A Gini score gives an idea of how good a split is by how mixed the classes are in the two groups created by the split. 

For classification problems, this cost function is often the **Gini index**, that calculates the purity of the groups of data created by the split point, whereas the worst case split that results in 50/50 classes in each group results in a Gini score of 1.0.

When building a binary tree, calculate it for every row and split the data accordingly and repeat this process recursively.


### Decision tree & RF

In [147]:
# load a csv file into list of lists - csv_reader creats an object to be read row by row 

def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
            
    return dataset

In [148]:
# convert string feature value to float - entries for each row are string value, removing the white sapce before and after the string
# and convert the entries to float values 
# str.strip() Returns a copy of the string with leading and trailing characters removed

def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


In [149]:
# convert string label value to integer - class label is documented in the last column

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)

    lookup = dict()
    # create mapping dict - string label to integer
    for i, value in enumerate(unique):
        lookup[value] = i
        
    for row in dataset:
        # dataset.replace(mapping)
        row[column] = lookup[row[column]]
    
    return lookup

In [150]:
# split data into k folds
# pop removes and returns the last item in the list

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset)/ n_folds)
    
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
        
    return dataset_split 

In [151]:
# calculate accuracy percentage

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
     
    return correct/ float(len(actual)) * 100
    

In [152]:
# CV in model selection 

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        # train_set is a list of K lists of folds 
        train_set = list(folds)
        train_set.remove(fold)
        # sum list of lists and empty list gives converts the list of lists to one list
        train_set = sum(train_set, [])
        
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
            
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        
    return scores 

In [153]:
# split dataset based on an attribute index and an attribute value 

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
            
    return left, right 

### Decision trees 

**test_split** 
- splits the dataset by using the given variable index and split value
- returns the splitted dataset as two lists 

**gni_index** 
- calculates the gini index for given two groups of data and their class labels 

**get_split** 
- finds the best split index and value by lopping through all those combination in the given dataset, i.e. using test_split on all combinations  
- calculates Gini index for the newly splitted groups
- if it is smaller than the previous one, update the dictionary that contains split index, value and groups 
- and returns the selected split index and value together with the resulting groups as two lists in a dictionary

**split(node, max_depth, min_size, n_features, depth)**
- splits the returned groups from get_split to two child nodes - node['left'] and node['right']
- and checks if the split can continue, if it can use get_split and split functions to continue the split, returns nothing 
- if it cannot, calculate the majority vote in that node and replace node['left'] and node['right'] with the voting results 

**build_tree** 
- starts from using get_split to create the initiall node expressed in a dictionary
- and continues the split using get_split and split as in the two blocks above 
- returns the dictionary of splits 

gini and entropy

http://www.learnbymarketing.com/481/decision-tree-flavors-gini-info-gain/

In [154]:
# calculate gini index for a given split results(attribute, value, child groups)

def gini_index(groups, classes):
    
    # count total number of samples in the two child groups 
    n_instances = float(sum([len(group) for group in groups]))
    
    # sum weighted gini index for each group
    gini = 0
    for group in groups:
        size = float(len(group))
        # avoid using zero as denominator 
        if size == 0:
            continue
        score = 0
        
        # score the split based on the score for the resulting class -- 1 - sumi (p_i **2)
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val)/ size
            score += p* p
            # weight group score by its relatvie size 
            gini += (1 - score) * (size/ n_instances)

    return gini

In [255]:
# select the best split point for a dataset and return the index, value and child groups 

def get_split(dataset, n_features):
    
    class_values = list(set(row[-1] for row in dataset)) # collect all the class labels in training dataset 
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    features = list()
    while len(features) < n_features:
        # get all the feature index
        index = randrange(len(dataset[0]) - 1)
        if index not in features:
            features.append(index)
    
    # gready search through all the feature index and value combination
    for index in features:
        for row in dataset:
            # split first then calcualte Gini
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            # update split when gini is smaller 
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups

    return {'index': b_index, 'value': b_value, 'groups': b_groups}
                

In [165]:
# returns a terminal node value - majority vote

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    
    return max(set(outcomes), key = outcomes.count)

In [333]:
# create child splits for a node or make terminal if conditions are met 

def split(node, max_depth, min_size, n_features, depth):
    
    print ({node['index'], node['value']})

    # create child splits for a node
    left, right = node['groups']
    # change 'groups' to 'left' and 'right'
    del(node['groups'])
    
    # check for a no split - when all the feature index and value combination has been checked 
    if not left or right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth + 1)
       
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else: 
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth + 1)
    

In [262]:
# build a decision tree

def build_tree(train, max_depth, min_size, n_features):
    # find the first best split and continue the split 
    root = get_split(train, n_features)
    
    # this root is passed as a node for the following splits and it is a dictionary of selected split index, value and groups 
    split(root, max_depth, min_size, n_features, 1)
    
    return root

In [168]:
# make a prediction with a decision tree

def predict(node, row):
    # predictions are made at the end nodes 
    # returns the predicted group the row belongs to need to use max(group) to find the class label later 
    
    if row[node['index']] < node['value']:
    # due to row[index] < value from test_split    
        
        # isinstance Returns a Boolean stating whether the object is an instance or subclass of another object
        # when conditions are not met, splits have to continue then the returns from split function is a dictionary 
        # of the following split results instead of that when terminate function is called 
        if isinstance(node['left'], dict): 
            return predict(node['left'], row)
        else:
            return node['left'] # left group
        
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
        

In [169]:
# create a random subsample from the dataset with replacement 

def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset)* ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
        
    return sample 

In [170]:
# make a prediction with a list of bagged trees 

def baggging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    
    return max(set(predictions), key = predictions.count)

In [171]:
# random forest algorithm

def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        
        # build_tree returns the tree definition dictionary {'index': b_index, 'value': b_value, 'groups': b_groups}
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
        
    # tree['index']
    predictions = [baggging_predict(trees, row) for row in test]
    
    return(predictions)

### Main

In [188]:
# test the random forest algorithm

seed(2)
filename = 'sonar.csv'
dataset = load_csv(filename)

# convert string attributes to integers 
for i in range(0, len(dataset[0]) - 1):
    str_column_to_float(dataset, i)
    
# convert class column to integers 
str_column_to_int(dataset, len(dataset[0]) - 1)

# algorithm evaluation 
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1
n_features = int(sqrt(len(dataset[0]) - 1))

for n_trees in [5, 100, 1000]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 5
Scores: [48.78048780487805, 21.951219512195124, 48.78048780487805, 31.70731707317073, 53.65853658536586]
Mean Accuracy: 40.976%
Trees: 100
Scores: [65.85365853658537, 51.21951219512195, 56.09756097560976, 41.46341463414634, 51.21951219512195]
Mean Accuracy: 53.171%
Trees: 1000
Scores: [36.58536585365854, 53.65853658536586, 60.97560975609756, 34.146341463414636, 46.34146341463415]
Mean Accuracy: 46.341%


In [337]:
# parameters 
n_folds = 5
max_depth = 1
min_size = 1
sample_size = 1
# 7 features
n_features = int(sqrt(len(dataset[0]) - 1))
n_trees = 5


# train test split 
folds = cross_validation_split(dataset, n_folds)



train_set = list(folds)
train_set.remove(folds[0])
# sum list of lists and empty list gives converts the list of lists to one list
train_set = sum(train_set, [])

test_set = list()
for row in folds[0]:
    row_copy = list(row)
    test_set.append(row_copy)
    row_copy[-1] = None
    
train = train_set
test = test_set



root = random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features)

# random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features)
trees = list()
for i in range(n_trees):
    sample = subsample(train, sample_size)

    # build_tree returns the tree definition dictionary {'index': b_index, 'value': b_value, 'groups': b_groups}
    tree = build_tree(sample, max_depth, min_size, n_features)
    trees.append(tree)

# tree['index']
predictions = [baggging_predict(trees, row) for row in test]

{0.203, 43}
{0.2128, 10}
{0.9085, 27}
{0.1556, 11}
{0.1645, 12}
{0.161, 10}
{0.1989, 10}
{0.2154, 11}
{0.2087, 11}
{0.1171, 5}


In [342]:
trees[0]

{'index': 10, 'left': 1, 'right': 1, 'value': 0.161}

In [341]:
trees

[{'index': 10, 'left': 1, 'right': 1, 'value': 0.161},
 {'index': 10, 'left': 0, 'right': 0, 'value': 0.1989},
 {'index': 11, 'left': 0, 'right': 0, 'value': 0.2154},
 {'index': 11, 'left': 0, 'right': 0, 'value': 0.2087},
 {'index': 5, 'left': 1, 'right': 1, 'value': 0.1171}]